In [1]:
import time

In [4]:
%timeit _=time.time()

68.1 ns ± 1.33 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [1]:
from udp_server import UDPServer
local_address=("127.0.0.1", 33300) # （ipv4/6 local address, local port)
remote_address=("127.0.0.1", 33301) #（ipv4/6 remote address, remote port)
server = UDPServer(local_address,remote_address)

('127.0.0.1', 33301)


In [13]:
# server.receive(2,False,True)

In [9]:
import socket
local_address=("127.0.0.1", 33300) # （ipv4/6 local address, local port)

(recv_family, type, proto, canonname, local_address) = socket.getaddrinfo(*local_address)[0]
recv_sock = socket.socket(family=recv_family, type=socket.SOCK_DGRAM)
%timeit recv_sock.settimeout(0)  # timeout immediately

1 µs ± 3.48 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [5]:
# import socket
# from io import BytesIO
# buffer = BytesIO()
# s1, s2 = socket.socketpair()
# b1 = bytearray(b'----')
# # s1.send(b'Mary had a little lamb')
# s2.settimeout(0.1)
# try:
#     s2.recv(100)
# except Exception as e:
#     print(type(e).__name__)

In [62]:
hasattr("test", "__len__")

True

In [60]:
np.array("test")

array('test', dtype='<U4')

In [1]:
from flexipod_env import FlexipodEnv
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
%config InlineBackend.figure_format = 'retina'
%load_ext line_profiler

env = FlexipodEnv()
# _ = env.reset()
# obs,_,_,_ = env.step(env.action_space.sample())

In [2]:
# pulley.server.send(pulley.packer.pack([[100],[1],[0]]))

In [1]:
from real_env.pulley_control import PulleyControl

In [2]:
pulley = PulleyControl()

('192.168.137.123', 32002)


In [3]:
self = pulley

In [2]:
self.send(self.reset_pos, 10)

In [4]:
self.reset()

In [4]:
self.move(-600,10)

In [8]:
pulley.setResetPos()

In [3]:
reset_pos = 'auto'
np.isscalar(reset_pos)

True

In [5]:
pulley.receive()
pulley.reset_pos = pulley.pos

(798.1256103515625, (1114.1044921875,), (0.0,), (0.10748291015625,))


In [53]:
pulley.send(pulley.reset_pos+-100,1)

In [115]:
fc.reset()

In [113]:
pulley.send(pulley.reset_pos +150,4)
time.sleep(1)
pulley.send(pulley.reset_pos+50,4)
time.sleep(0.5)
pulley.send(pulley.reset_pos,4)
fc.reset()
pulley.send(pulley.reset_pos,0.1)

t_start = time.time()
t_druation =5


dt = 0.01
dp = 20
action = np.zeros(12)

while time.time()-t_start<t_druation:
    pulley.receive()
    dpos = abs(pulley.pos - pulley.reset_pos)
    if dpos>40:
        k = 2
#         k = np.clip(0.2+dpos*0.1,0.2,4)
    else:
        k=1
    pulley.send(pulley.reset_pos,k)
    
#     action=[random.uniform(-2,2) for i in range(12)]
#     fc.send(action,vel_control = 1)

    fc.receive()
    action_ = (np.random.random(12)-0.5)*dp*dt
    action = 0.95*action+0.05*action_
#     print(action)
    for i in range(12):
        if fc.joint_pos[i]<fc.joint_pos_limit[i,0]:
            action[i] = 0
        elif fc.joint_pos[i]>fc.joint_pos_limit[i,1]:
            action[i] = 0
#     print(action)
#     action[np.logical_and(action<0,fc.joint_pos<fc.joint_pos_limit[:,0])]=0
#     action[np.logical_and(action>0,fc.joint_pos>fc.joint_pos_limit[:,1])]=0
#     print(action)
#     print(action,fc.joint_pos)
    action_p = action+fc.joint_pos
    fc.send(list(action_p),0)
#     time.sleep(dt)

fc.reset()
pulley.send(pulley.reset_pos +100,4)
time.sleep(1)
pulley.send(pulley.reset_pos+50,4)
time.sleep(0.5)
pulley.send(pulley.reset_pos,4)


KeyboardInterrupt: 

In [98]:
pi = np.pi

p_f_01 = pi/4 # front arm 01

p_bl_b0 = 0
p_br_b0 = 0

p_bl_01 = 0
p_br_01 = 0

n0 = pi/25 # body incline
n1 = pi/6
n2 = 0
n3 = pi/180 # hip left-right


p_bl_12 = p_bl_b0*1.1
p_br_12 = p_br_b0*1.1
joint_pos = np.array([
# front left
    0,
    -p_f_01,
    0,
# front right
    0,
    p_f_01,
    0,
# back left
    -p_bl_b0-n0,
    -pi/2+p_bl_01,
    -p_bl_12,
#back right
     p_br_b0+n0,
     pi/2-p_br_01,
     p_br_12,
])

while True:
    fc.receive()
    delta_p = 3*(joint_pos-fc.joint_pos)
    if np.linalg.norm(delta_p)>4e-2:
#         env.step(delta_p)
        fc.send(list(delta_p),1)  
#             time.sleep(0.0001)
    else:
        break
            


for p_bl_b0,p_br_b0,p_bl_01,p_br_01 in zip([n2,n1,n2]*999,[n1,n2,n1]*999,[-n3,n3,-n3]*999,[n3,-n3,n3]*999):
    
# for p_bl_01,p_br_01 in zip([0,pi/18,0]*50,[pi/18,0,pi/18]*50):

    
    p_bl_12 = p_bl_b0*1.1

    p_br_12 = p_br_b0*1.1

    
    
    joint_pos = np.array([
    # front left
        0,
        -p_f_01,
        0,
    # front right
        0,
        p_f_01,
        0,
    # back left
        -p_bl_b0-n0,
        -pi/2+p_bl_01,
        -p_bl_12,
    #back right
         p_br_b0+n0,
         pi/2-p_br_01,
         p_br_12,
    ])

    while True:
        delta_p = 5*(joint_pos-fc.joint_pos)
        if np.linalg.norm(delta_p)>1e-1:
            fc.receive()
#             fc.send(list(delta_p),1)     
            fc.send(list(joint_pos),0)     

#             env.step(delta_p)
    #             time.sleep(0.0001)
        else:
            break


NameError: name 'fc' is not defined

In [88]:
fc.reset()

UnboundLocalError: local variable 'data_unpacked' referenced before assignment

In [4]:
joint_pos = np.array([
    # front left
        0,
        0,
        0,
    # front right
        0,
        0,
        0,
    # back left
        0,
        0,
        0,
    #back right
        0,
        0,
        0,
    ])

fc.send(joint_pos.tolist(),0)

In [139]:
joint_pos = np.array([
    # front left
        0,
        0,
        0,
    # front right
        0,
        0,
        0,
    # back left
        0,
        0,
        0,
    #back right
        0,
        0,
        0.5,
    ])

fc.send(joint_pos.tolist(),0)
time.sleep(0.2)
print(fc.receive()[2][0:24])
print(fc.receive()[9])
np.set_printoptions(precision=3)
fc.joint_pos

(1.0, 0.0, 1.0, 0.0, 1.0, 2.1855694143368964e-08, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.1984408047283068e-05, 1.0, 0.0, 1.0, 0.0, 0.8775818347930908, 0.47942686080932617)
(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.1984408047283068e-05, 0.0, 0.0, 0.5000014901161194, 0.0, 2.1855694143368964e-08, 1.1984408047283068e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1855694143368964e-08, 0.0, 0.0, 0.5000014901161194, 0.0, 2.1855694143368964e-08, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1855694143368964e-08, 0.0, 0.0, 0.5000014901161194, 0.0, 2.1855694143368964e-08, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1855694143368964e-08, 0.0, 0.0, 0.49998947978019714, 0.0, 2.1855694143368964e-08, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1855694143368964e-08, 0.0, 0.0, 0.5000014901161194)


array([0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 1.198e-05, 0.000e+00, 0.000e+00, 5.000e-01],
      dtype=float32)

In [44]:
joint_pos = np.array([
    # front left
        0,
        0,
        0,
    # front right
        0,
        0,
        0,
    # back left
        0,
        0,
        0,
    #back right
        0,
        0,
        0,
    ])

fc.send(joint_pos.tolist(),1)

In [19]:
fc.joint_pos

array([ 7.1906448e-05, -1.2566611e+00,  2.3968816e-05, -7.1920949e-05,
        1.2568887e+00,  0.0000000e+00,  7.1906448e-05, -1.2566611e+00,
       -4.7959882e-05, -4.7959882e-05,  1.2569007e+00,  0.0000000e+00],
      dtype=float32)

In [103]:
pulley.send(pulley.reset_pos,4)


In [36]:
pulley.send(pulley.reset_pos +200,4)
fc.reset()


In [ ]:
pulley.send(pulley.reset_pos,0.01)


In [20]:
# pulley.receive()
pulley.move(100,k=1)

x = self.packer.pack([[10]])
self.server.send(x)

In [29]:
%timeit _=pulley.receive(max_attempts=1000000)

1.04 ms ± 9.81 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [1]:
import msgpack
import numpy as np
import time
from typing import Union

# fix relative import
import sys
import os
# SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
# sys.path.append(os.path.dirname(SCRIPT_DIR))
# # import from parent directory
from udp_server import UDPServer


class FlexipodControl:
    def __init__(s,
                 local_address = ("192.168.137.1",32005),
                 remote_address = ("192.168.137.130",32004)
                 ):
        s.server = UDPServer(local_address=local_address,remote_address=remote_address)
        # reset data packed # https://msgpack-python.readthedocs.io/en/latest/api.html#msgpack.Packer
        s.packer = msgpack.Packer(use_single_float=True, use_bin_type=True)
        
        s._set_reset_pos() # set the joint reset position
        
        pi = np.pi
        s.joint_pos_limit = np.array([ # joint pos limit
                # front left
                [-pi*0.8,pi*0.8], # 0
                [-pi/2.2,pi/2.2], # 1
                [-pi/2,pi/2], # 2
                # front right
                [-pi*0.8,pi*0.8], # 3
                [-pi/2.2,pi/2.2], # 4
                [-pi/2,pi/2], # 5
                # back left
                [-pi/4,pi/4], # 6, reduced range
                [-pi/2-pi/8,-pi/2+pi/4], # 7, reduced range
                [-pi/2,pi/2], # 8
                # back right
                [-pi/4,pi/4], # 9 , reduced range
                [pi/2-pi/4,pi/2+pi/8], # 10, reduced range
                [-pi/2,pi/2], # 11
            ], dtype=np.float32)
        
        
        from phase_indication import phaseIndicatorPair
        s.phase_indicator_pair_0 = phaseIndicatorPair(a=0.3, b=0.7, s=0.05, t0=0.25, t1=0.75, ys = -1., y0=0.)
        s.phase_indicator_pair_1 = phaseIndicatorPair(a=0.25, b=0.75, s=0.05, t0=0.25, t1=0.75, ys = 2., y0=-1.)
        
        s.t_normalized =0
        
        s.timeout = 2 # timeout in [second]
#         s.reset()
        
    def _set_reset_pos(s):
        """ set the reset position"""
        pi = np.pi

        p_f_b0 = -pi/6 # font arm body-0
        p_f_01 = pi/3 # front arm 01
        p_f_12 = -pi/3 # front arm 12

        p_bl_b0 = pi/12
        p_br_b0 = pi/12

        p_bl_01 = -p_bl_b0*2
        p_br_01 = -p_br_b0*2

        p_bl_12 = p_bl_b0
        p_br_12 = p_br_b0

        # n0 = pi/20 # body incline
        n0 = 0 # body incline

        s.joint_pos_reset = np.array([ # reset joint position
            # front left
            -p_f_b0,      # 0
            -p_f_01,      # 1
            -p_f_12,      # 2
            # front right
            p_f_b0,      # 3
            p_f_01,      # 4
            p_f_12,      # 5
            # back left
            -p_bl_b0-n0,   # 6
            -p_bl_01,      # 7
            -p_bl_12,      # 8
            #back right
             p_br_b0+n0,   # 9
             p_br_01,      # 10
             p_br_12,      # 11
        ])

    def reset(s):
        s.send(s.joint_pos_reset,vel_control=0)
        s.receive()
        while(np.linalg.norm(s.joint_pos-s.joint_pos_reset,ord=1)>0.2):
            s.send(s.joint_pos_reset,vel_control=0)
            s.receive()
            
    def receive(s,verbose=False):
        try:
            data = s.server.receive(timeout=s.timeout,clear_buf=True,newest=False) # max of timeout second
            # print(len(data))
            data_unpacked = msgpack.unpackb(data,use_list=False)
            joint_pos = data_unpacked[1][0][1]
            s.joint_pos = np.arctan2(joint_pos[1::2],joint_pos[::2],dtype=np.float32)
        except TimeoutError:
            pass
        return data_unpacked
    
    def send(s,action, vel_control = 0):
        packed = s.packer.pack([action.tolist() if type(action) is np.ndarray else action,vel_control])
        #packed = msgpack.packb([action,[vel_control]])
        s.server.send(packed)
    
#     def step(s,action, vel_control = 1):

    def exampleAction(s):
        pi = np.pi
        p_f_b0 = pi/24 # font arm body-0
        p_f_01 = pi/3 # front arm 01
        p_f_12 = -pi/12 # front arm 12
        p_b0_base = pi/8 # back leg body-0
        while True:
            # indicator for foot force
            i0_0,i0_1= s.phase_indicator_pair_0.get(s.t_normalized)
            # indicator for foot forward displacemnt relative to body, input to the example gait generator
            i1_0,i1_1= s.phase_indicator_pair_1.get(s.t_normalized)

            i0 = i1_0 # use phase_indicator_pair_1
            i1 = i1_1

            p_bl_b0 =p_b0_base+i0*0.08
            p_br_b0 =p_b0_base+i1*0.08

            p_bl_01 = -p_bl_b0*2
            p_br_01 = -p_br_b0*2

            p_bl_12 = p_bl_b0
            p_br_12 = p_br_b0

            joint_pos = np.array([
                # front left
                    -p_f_b0,
                    -p_f_01,
                    -p_f_12,
                # front right
                    p_f_b0,
                    p_f_01,
                    p_f_12,
                # back left
                    -p_bl_b0 - i0*0.08 ,
                    -p_bl_01,
                    -p_bl_12,
                #back right
                    p_br_b0 + i1*0.08,
                    p_br_01,
                    p_br_12,
            ])

#             delta_p = 1*(joint_pos-s.joint_pos)
#             yield delta_p
            yield joint_pos

            
#     def act(s,action=None):
#         """do action"""
#         if action is not None:
#             # map action -> action*max_acton
#             cmd_action = np.asarray(action,dtype=np.float32)
# #             if s.normalize:
# #                 cmd_action = s.toRawAction(cmd_action)
#             # # position difference control
#             cmd_action += s.joint_pos # the actual position
            
#             # if len(s.cd.getContactPoints(cmd_action))>0:
#             step_cmd_b = s.packer.pack([s.UDP_MOTOR_POS_COMMEND,time.time(),cmd_action.tolist()])
#             # step_cmd_b = s.packer.pack([s.UDP_MOTOR_VEL_COMMEND,time.time(),cmd_action.tolist()])
#             num_bytes_send = s.server.send(step_cmd_b)
        
            

if 'fc' in globals():
    fc.server.close()
    del fc

fc = FlexipodControl()
# s = fc

('192.168.137.130', 32004)


In [2]:
fc.reset()

timeout: timed out

In [6]:
data = fc.server.receive()
data_unpacked = msgpack.unpackb(data,use_list=False)

In [7]:
fc.receive()

(114514,
 ((1638798826244203,
   (1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0),
   (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
   (-1.96380615234375,
    -1.5106201171875,
    -1.96380615234375,
    1.66168212890625,
    1.20849609375,
    -3.021240234375,
    -1.96380615234375,
    -4.833984375,
    -4.5318603515625,
    4.07867431640625,
    8.1573486328125,
    6.3446044921875),
   (0.9909006357192993,
    -0.13449877500534058,
    0.00506959855556488,
    0.10407036542892456,
    0.7895259857177734,
    0.6048288345336914),
   (0.0, 0.0, 0.0),
   (0.97607421875, -0.01171875, 0.01171875),
   (-45.6207275390625, -90.076904296875, 105.18310546875),
   (0.0, 0.0)),
  (1638798826231972,
   (1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
   

In [7]:
imu1=np.array([[0,0,-1],
              [0,1,0],
              [1,0,0]])
imu2=np.array([[1,0,0],
              [0,-1,0],
              [0,0,-1]])
# cali=np.array([[-0.0562617, 0.907498, 0.416272],
# [0.336706, -0.375262, 0.863601],
# [0.939927, 0.188749, -0.284447]])
cali=np.array([[-0.127552, -0.217422, 0.967708],
[-0.0889155, -0.969242, -0.229486],
[0.987838, -0.115316, 0.104297]])
w_c=np.linalg.inv(cali@imu1@imu2)
r_imu_robot=imu1@imu2
w_c
#r_imu_robot

array([[ 0.96770745, -0.2294865 ,  0.10429657],
       [ 0.21742137,  0.96924202,  0.11531564],
       [-0.12755244, -0.08891575,  0.98783853]])

In [11]:
fc.send(joint_pos.tolist(), vel_control = 0)

In [37]:
exp = fc.exampleAction()
while(1):
    gait_frequency = 1.5
    episode_start_time = 0
    t = time.time()
    fc.t_normalized = ((t-episode_start_time)*gait_frequency)%1
    action = next(exp)
#     print(action)
    time.sleep(0.01)
    fc.send(action.tolist())

KeyboardInterrupt: 

In [2]:
fc.receive()

NameError: name 'fc' is not defined

In [1]:
import time
import datetime
import msgpack
import numpy as np
from udp_server import UDPServer 
local_address = ("192.168.137.1",32005)
remote_address = ("192.168.137.130",32004)
server = UDPServer(local_address=local_address,remote_address=remote_address)
packer = msgpack.Packer(use_single_float=True, use_bin_type=True)


('192.168.137.130', 32004)


In [4]:
data = server.receive(clear_buf=True, newest=True)
data_unpacked = msgpack.unpackb(data,use_list=False)

datetime_time_from_recv = datetime.datetime.fromtimestamp(data_unpacked[1][0][0]/1000000)
datetime_time_sys = datetime.datetime.fromtimestamp(time.time())

print(datetime_time_from_recv)
print(datetime_time_sys)
print(time.time() - data_unpacked[1][0][0]/1000000)

joint_pos = data_unpacked[1][0][1]
joint_pos = np.arctan2(joint_pos[1::2],joint_pos[::2],dtype=np.float32)
print(joint_pos)

2021-12-05 19:22:14.664974
2021-12-05 19:22:15.348009
0.6839823722839355
[ 0.0000000e+00 -1.1984408e-05  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  1.1984408e-05]


In [6]:
joint_pos_cmd = np.zeros(12)
#joint_pos_cmd[3] = np.pi/6
packed = packer.pack([joint_pos_cmd.tolist(),0])
server.send(packed)

63

In [6]:
while 1:
    data = server.receive(clear_buf=True, newest=True)
    data_unpacked = msgpack.unpackb(data,use_list=False)
    print(data_unpacked[1][0][-1])

(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)

(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)

(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)

(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)

(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)

(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)

(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)

(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)

KeyboardInterrupt: 

In [10]:
data_unpacked[1][0][-1]

(0.0, 0.0)

In [93]:
# data_unpacked

In [188]:
%timeit _ = server.receive(clear_buf=False, newest=False)

4.08 ms ± 58.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [107]:
%timeit _ = server.receive()

4.38 ms ± 219 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [108]:
1000/3.77

265.2519893899204

In [5]:
fc.send(fc.joint_pos_reset,vel_control=0)

In [7]:
import sys

data = fc.server.receive()
data_unpacked = msgpack.unpackb(data,use_list=False)
print(sys.getsizeof(data))
print(len(data_unpacked))

1995
2


In [4]:
data_unpacked

(114514,
 ((2395.556640625,
   (1.0,
    0.0,
    0.30900561809539795,
    -0.9510602355003357,
    1.0,
    0.0,
    1.0,
    0.0,
    0.30900561809539795,
    0.9510602355003357,
    1.0,
    0.0,
    1.0,
    0.0,
    -4.371138828673793e-08,
    -1.0,
    1.0,
    0.0,
    1.0,
    0.0,
    -4.371138828673793e-08,
    1.0,
    1.0,
    0.0),
   (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
   (1054.7109375,
    1055.98388671875,
    1055.98388671875,
    1.30517578125,
    0.064453125,
    1055.61328125,
    1054.4208984375,
    0.08056640625,
    0.24169921875,
    1.04736328125,
    0.12890625,
    1055.548828125),
   (-0.0415496751666069,
    0.9984591603279114,
    0.03678318113088608,
    0.12963511049747467,
    0.04189107567071915,
    -0.9906764030456543),
   (-3.731563629116863e-05, 0.00012793931819032878, -6.396965909516439e-05),
   (-0.02153540402650833, 0.13878372311592102, 9.56650447845459),
   (0.009553015232086182, 0.017967307940125465, -0.149601489305

In [122]:
a = np.arange(24)

In [123]:
a[1::2]

array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23])

In [124]:
a[::2]

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22])

In [121]:
print(fc.receive()[1][0][0]/1000)
print(time.time())
print(time.time())
fc.receive()

1635227402.24
1635227418.147826
1635227418.147826


(114514,
 ((1635227402240.0,
   (1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    0.0,
    1.0,
    1.1984408047283068e-05),
   (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
   (1055.87109375,
    1055.9677734375,
    1055.7099609375,
    0.2900390625,
    1055.806640625,
    1055.85498046875,
    1054.88818359375,
    1054.8720703125,
    1055.162109375,
    0.56396484375,
    0.966796875,
    0.82177734375),
   (-0.915357232093811,
    0.22264105081558228,
    0.3354880213737488,
    -0.3459323048591614,
    -0.8612274527549744,
    -0.3723144829273224),
   (0.0, 0.0, 0.0),
   (0.9765625, -0.0078125, -0.0166015625),
   (-154.7479248046875, -89.9725341796875, 36.5240478515625),
   (0.0, 0.0),
   (0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    1.1984408047283068e-05)),
  (163522740

In [72]:
action=[random.uniform(-0.01,0.01) for i in range(12)]
dd=fc.packer.pack([action,[1]])
da=msgpack.unpackb(dd)
print(da)

[[0.0033690696582198143, 0.008548925630748272, -0.0089388657361269, -0.008423020131886005, 0.007582091260701418, 0.0059034936130046844, 0.001196678844280541, -0.004834413528442383, 0.0028027594089508057, 0.005315433256328106, -0.0032196613028645515, -0.006349790841341019], [1]]


In [2]:
%timeit _=fc.receive()

4.03 ms ± 76.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
fc.joint_pos_limit.T

array([[-2.5132742, -1.4279966, -1.5707964, -2.5132742, -1.4279966,
        -1.5707964, -0.7853982, -1.9634954, -1.5707964, -0.7853982,
         0.7853982, -1.5707964],
       [ 2.5132742,  1.4279966,  1.5707964,  2.5132742,  1.4279966,
         1.5707964,  0.7853982, -0.7853982,  1.5707964,  0.7853982,
         1.9634954,  1.5707964]], dtype=float32)

In [29]:
import random
while 1:

    action=[random.uniform(-1,1) for i in range(12)]
    fc.send(action,vel_control = 1)

KeyboardInterrupt: 

In [30]:
fc.reset()

In [90]:
fc.reset()

In [89]:
action = np.zeros(12)

dt = 0.01
dp = 10
for k in range(100000):
    fc.receive()
    action_ = (np.random.random(12)-0.5)*dp*dt
    action = 0.9*action+0.1*action_
#     print(action)
    for i in range(12):
        if fc.joint_pos[i]<fc.joint_pos_limit[i,0]:
            action[i] = 0
        elif fc.joint_pos[i]>fc.joint_pos_limit[i,1]:
            action[i] = 0
#     print(action)
#     action[np.logical_and(action<0,fc.joint_pos<fc.joint_pos_limit[:,0])]=0
#     action[np.logical_and(action>0,fc.joint_pos>fc.joint_pos_limit[:,1])]=0
#     print(action)
#     print(action,fc.joint_pos)
    action_p = action+fc.joint_pos
    fc.send(list(action_p),0)
#     time.sleep(dt)

KeyboardInterrupt: 

In [2]:
data_unpacked = fc.receive()

In [7]:
pos=fc.joint_pos_reset
fc.send(list(pos))
time.sleep(1)
data_unpacked = fc.receive()

pos = self.joint_pos
pos[10]+=0.01
fc.send(pos.tolist())
data_unpacked = fc.receive()


In [47]:
pos

array([ 0.       , -1.256649 ,  0.       ,  0.       ,  1.2566371,
        0.       ,  0.       , -1.256649 ,  0.       ,  0.       ,
        1.2666371,  0.       ], dtype=float32)

In [16]:
fc.joint_pos_reset>fc.joint_pos_limit[:,0]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [17]:
fc.joint_pos_reset<fc.joint_pos_limit[:,1]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])